# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [5]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import nltk
nltk.download(['punkt', 'wordnet'])
import re
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jsong\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\jsong\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('InsertTableName', 'sqlite:///InsertDatabaseName.db')  
X = df.loc[:,'message'] 
Y = df.iloc[:, 4:] 

In [3]:
Y.related.value_counts()

1    20093
0     6122
Name: related, dtype: int64

### 2. Write a tokenization function to process your text data

In [8]:
def tokenize(text):
    
    #: tokenize text
    tokens= word_tokenize(text)
    
    #: initiate Lemmatizer
    lemmatizer = WordNetLemmatizer()
    
    #: iterate through each token
    clean_tokens = []
    for tok in tokens:
        
        #: Lemmatize, normalize case, and remove Leading/trailing wite space
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
    
    return clean_tokens



### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [9]:
pipeline = Pipeline([
    ('vector', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y)
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vector', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        str...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [11]:
y_predict = pipeline.predict(X_test)

In [12]:
from sklearn.metrics import classification_report

print(classification_report(y_test.values, y_predict, target_names=Y.columns.values))

                        precision    recall  f1-score   support

               related       0.83      0.93      0.88      5016
               request       0.84      0.39      0.54      1176
                 offer       0.00      0.00      0.00        27
           aid_related       0.73      0.51      0.60      2702
          medical_help       0.56      0.08      0.14       516
      medical_products       0.70      0.05      0.10       312
     search_and_rescue       0.40      0.02      0.04       193
              security       0.38      0.02      0.04       126
              military       0.47      0.04      0.07       220
           child_alone       0.00      0.00      0.00         0
                 water       0.91      0.16      0.27       425
                  food       0.88      0.40      0.55       739
               shelter       0.88      0.21      0.34       593
              clothing       0.83      0.05      0.09       102
                 money       0.50      

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [13]:
pipeline.get_params()

{'memory': None,
 'steps': [('vector',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x0000026623E45BF8>,
           vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, r

In [14]:
from sklearn.model_selection import GridSearchCV

parameters ={
        'clf__estimator__n_estimators': [10, 50],
        #'clf__estimator__min_samples_split': [2, 5, 10],
        #'clf__estimator__min_samples_leaf': [1, 2, 4],
        'clf__estimator__max_depth': [20, None]
    }

In [15]:
cv = GridSearchCV(estimator=pipeline, param_grid=parameters, cv=3, n_jobs = -1, verbose = 2)

In [ ]:
cv.fit(X_train, y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
y_predict = cv.predict(X_test)

In [ ]:
print(classification_report(y_test.values, y_predict, target_names=Y.columns.values))

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
from sklearn.tree import DecisionTreeClassifier

pipeline = Pipeline([
    ('vector', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(DecisionTreeClassifier()))
])

In [ ]:
parameters ={
        'clf__estimator__min_impurity_decrease': [0.0]
    }

pipeline.get_params()

In [ ]:
cv = GridSearchCV(estimator=pipeline, param_grid=parameters, cv=3)

In [ ]:
cv.fit(X_train, y_train)

In [ ]:
y_predict = cv.predict(X_test)

In [ ]:
print(classification_report(y_test.values, y_predict, target_names=Y.columns.values))

### 9. Export your model as a pickle file

In [ ]:
model_filepath = 'classifier.pickle'

with open(model_filepath, 'wb') as model_file:
        pickle.dump(model, model_file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.